<a href="https://colab.research.google.com/github/UmarIgan/Machine-Learning/blob/master/microsoft_Phi1_5_wealth_alpaca_lora_fine_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fine Tuning with Microsoft phi
motivation: https://www.youtube.com/watch?v=R8CKx5yNEDo

# Installing Dependencies

- Installing libraries that will be used

In [1]:
! pip install accelerate transformers einops datasets peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 18.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 27.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.6/85.6 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.2 MB/s eta 0:00:00


- Login to huggingface for pushing model, get token wıth push feature**

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# Imports

In [2]:
import torch
from datasets import load_dataset, Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, DataCollatorForLanguageModeling, BitsAndBytesConfig
from peft import LoraConfig, get_peft_model
import os

# Training&Fine-Tuning

- Good result seem with phi so i use it here.

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

setting up config for [bitsandbytes](https://colab.research.google.com/drive/1b0GMx_Cp0huYVi6OOmciZ1smzkKYS7DP#scrollTo=L7hYn0WwEI68&line=1&uniqifier=1) for quantization.

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/phi-1_5",
    device_map={"":0},
    trust_remote_code=True,
    quantization_config=bnb_config
)

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


- lora for accelerating the fine-tuning

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["Wqkv", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 4,718,592 || all params: 1,422,989,312 || trainable%: 0.3315971497613047


In [ ]:
def add_prefix(example):#function for creating instructor
    example["text"] = ('question: ' + ' '.join([str(item) for item in example['instruction']]) + " answer: "+  example["output"])
    return example


In [ ]:
def tokenize(sample):
    model_inps =  tokenizer(sample["text"], padding=True, truncation=True, max_length=512)
    return model_inps

-Our dataset spesified in investment domain which can be a good use case for fine tuning.

In [ ]:
data = load_dataset("gbharti/wealth-alpaca_lora",  split="train[:40000]")#we will left some data for testing fine tunde model
data =  data.map(add_prefix)
tokenized_data = data.map(tokenize, batched=True, desc="Tokenizing data", remove_columns=data.column_names)
tokenized_data

Map:   0%|          | 0/40000 [00:00<?, ? examples/s]

Tokenizing data:   0%|          | 0/40000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 40000
})

In [ ]:
training_arguments = TrainingArguments(
        output_dir="phi-1_5-finetuned-wealth-alpaca_lora",
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        learning_rate=2e-4,
        lr_scheduler_type="cosine",
        save_strategy="epoch",
        logging_steps=100,
        max_steps=1000,
        num_train_epochs=1,
        push_to_hub=True
    )

In [ ]:
trainer = Trainer(
    model=model,
    train_dataset=tokenized_data,
    args=training_arguments,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
trainer.train()

You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
100,2.547600
200,2.525800
300,2.421800
400,2.412900
500,2.498500
600,2.380500
700,2.389000
800,2.483300
900,2.409700
1000,2.515500


TrainOutput(global_step=1000, training_loss=2.4584663543701173, metrics={'train_runtime': 372.993, 'train_samples_per_second': 10.724, 'train_steps_per_second': 2.681, 'total_flos': 8770631329972224.0, 'train_loss': 2.4584663543701173, 'epoch': 0.1})

In [ ]:
trainer.push_to_hub()

'https://huggingface.co/umarigan/phi-1_5-finetuned-wealth-alpaca_lora/tree/main/'

-adding base model

In [ ]:
from peft import PeftModel
from transformers import AutoModelForCausalLM
import torch
model = AutoModelForCausalLM.from_pretrained("microsoft/phi-1_5", trust_remote_code=True, torch_dtype=torch.float32)
peft_model = PeftModel.from_pretrained(model, "umarigan/phi-1_5-finetuned-wealth-alpaca_lora", from_transformers=True)
model = peft_model.merge_and_unload()

MixFormerSequentialForCausalLM(
  (layers): Sequential(
    (0): Embedding(
      (wte): Embedding(51200, 2048)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (1): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwise_affine=True)
      (resid_dropout): Dropout(p=0.0, inplace=False)
      (mixer): MHA(
        (rotary_emb): RotaryEmbedding()
        (Wqkv): Linear(in_features=2048, out_features=6144, bias=True)
        (out_proj): Linear(in_features=2048, out_features=2048, bias=True)
        (inner_attn): SelfAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
        (inner_cross_attn): CrossAttention(
          (drop): Dropout(p=0.0, inplace=False)
        )
      )
      (mlp): MLP(
        (fc1): Linear(in_features=2048, out_features=8192, bias=True)
        (fc2): Linear(in_features=8192, out_features=2048, bias=True)
        (act): NewGELUActivation()
      )
    )
    (2): ParallelBlock(
      (ln): LayerNorm((2048,), eps=1e-05, elementwis

In [ ]:
model.push_to_hub("phi-1_5-finetuned-wealth-alpaca_lora_uu")

pytorch_model.bin:   0%|          | 0.00/5.67G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/umarigan/phi-1_5-finetuned-wealth-alpaca_lora_uu/commit/c254b34213c199ac6577c1ca17a446afd7241af1', commit_message='Upload MixFormerSequentialForCausalLM', commit_description='', oid='c254b34213c199ac6577c1ca17a446afd7241af1', pr_url=None, pr_revision=None, pr_num=None)

# Testing

In [10]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
model = AutoModelForCausalLM.from_pretrained("umarigan/phi-1_5-finetuned-wealth-alpaca_lora_uu", trust_remote_code=True, torch_dtype=torch.float32)
tokenizer = AutoTokenizer.from_pretrained("microsoft/phi-1_5", trust_remote_code=True)

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- configuration_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-1_5:
- modeling_mixformer_sequential.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


In [3]:
data_test = load_dataset("gbharti/wealth-alpaca_lora",  split="train[40000:]")#test data

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
inputs = tokenizer('''question: What a good CAGR tells for a stock? answer: ''', return_tensors="pt", return_attention_mask=False)
outputs = model.generate(**inputs, max_length=1048)
text = tokenizer.batch_decode(outputs)[0]
text

"question: What a good CAGR tells for a stock? answer: \nI think the CAGR is a good metric to use for stocks. It's a good way to compare the performance of different stocks over time. However, it's important to remember that the CAGR is just one metric and it doesn't tell the whole story. You should also look at other metrics, such as the dividend yield, the price-to-earnings ratio, and the dividend growth rate. \nThe dividend yield is a good metric to use for stocks that pay dividends. It's a good way to compare the dividend yield of different stocks. The price-to-earnings ratio is a good metric to use for stocks that are undervalued. It's a good way to compare the price-to-earnings ratio of different stocks. The dividend growth rate is a good metric to use for stocks that have a history of increasing dividends. It's a good way to compare the dividend growth rate of different stocks. \nIn addition to these metrics, you should also look at the company's financial statements, such as th